# Hola &#x1F600;

Soy **Hesus Garcia**, revisor de código de Triple Ten, y voy a examinar el proyecto que has desarrollado recientemente. Si encuentro algún error, te lo señalaré para que lo corrijas, ya que mi objetivo es ayudarte a prepararte para un ambiente de trabajo real, donde el líder de tu equipo actuaría de la misma manera. Si no puedes solucionar el problema, te proporcionaré más información en la próxima oportunidad. Cuando encuentres un comentario,  **por favor, no los muevas, no los modifiques ni los borres**. 

Revisaré cuidadosamente todas las implementaciones que has realizado para cumplir con los requisitos y te proporcionaré mis comentarios de la siguiente manera:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

</br>

**¡Empecemos!**  &#x1F680;


## LIBRERIAS

In [23]:
import pandas as pd
import numpy as np
import plotly_express as px
import random

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm
from catboost import CatBoostClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, classification_report, average_precision_score

np.random.seed(999)
random.seed(999)

In [2]:
#Logistic regression
#Random forest
# KNN

#Lightgbm
# CatBoost
# Red Neuronal

In [3]:
try:
    data_contract = pd.read_csv("final_provider/contract.csv")
    data_internet = pd.read_csv("final_provider/internet.csv")
    data_personal = pd.read_csv("final_provider/personal.csv")
    data_phone    = pd.read_csv("final_provider/phone.csv")
except:
    data_contract = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/final_provider/contract.csv")
    data_internet = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/final_provider/internet.csv")
    data_personal = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/final_provider/personal.csv")
    data_phone    = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/final_provider/phone.csv")



In [4]:
data_contract.set_index('customerID', inplace=True)
data_internet.set_index('customerID', inplace=True)
data_personal.set_index('customerID', inplace=True)
data_phone.set_index('customerID', inplace=True)


In [5]:
#juntar todas las tablas
full_data = data_contract.join([data_internet, data_personal, data_phone])


In [6]:
full_data["BeginDate"] = pd.to_datetime(full_data["BeginDate"], errors='coerce')
full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')

full_data['BeginDate_Year'] = full_data['BeginDate'].dt.year
full_data['BeginDate_Month'] = full_data['BeginDate'].dt.month

full_data['BeginDate_Year'] = full_data['BeginDate_Year'].astype("str")
full_data['BeginDate_Month'] = full_data['BeginDate_Month'].astype("str")

# Creación de la columna objetivo (target)
full_data["isActive"] = full_data["EndDate"].apply(
    lambda x: 0 if pd.isna(x) else 1
)

full_data = full_data.drop(columns=["BeginDate", "EndDate"], axis =1)

# Relleno de valores ausentes, al observar los datos faltantes es muy probable que todos sean por el tipo de plan contratado, no existen datos porque no han contratado un plan que pueda generar datos en esas características
full_data.fillna("No data", inplace=True)

# Reemplazamos strings vacios por ceros
full_data.replace({"MonthlyCharges": {" ": 0}, "TotalCharges": {" ": 0}}, inplace=True)




C:\Users\armod\AppData\Local\Temp\ipykernel_17888\1594772674.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')


In [8]:
full_data["isActive"].value_counts()

isActive
0    5174
1    1869
Name: count, dtype: int64

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Tu enfoque para unir los diferentes conjuntos de datos y crear la variable objetivo es claro y efectivo. La creación de características temporales a partir de las fechas es una excelente práctica, ya que te permite capturar patrones temporales que podrían ser críticos para predecir el comportamiento del usuario a lo largo del tiempo. Este tipo de ingeniería de características es clave para mejorar la precisión de los modelos predictivos.
</div>


## EVALUACION DE MODELOS

In [10]:
def scaler_ohe(df):
    scaler = StandardScaler()
    scaler.fit(df[numeric_columns])
    data_scaled = scaler.transform(df[numeric_columns])
    df[numeric_columns]= data_scaled
    df = pd.get_dummies(df, drop_first=True)
    
    return df

def find_scores(features_test, target_test, predictions, model):

    print(classification_report(target_test, predictions))
    print("Área bajo la curva ROC:", roc_auc_score(target_test, predictions))

    #Buscamos las probabilidades de que el valor objetivo sea 1, dependiendo el tipo de modelo es el metodo que elegimos
    try:
        probab_test = model.predict_proba(features_test)
        probab_one_test = probab_test[:, 1]
        fpr, tpr, thresholds = roc_curve(target_test, probab_one_test)  
        
    except:
        probab_one_test = model.predict(features_test)
        fpr, tpr, thresholds = roc_curve(target_test, probab_one_test) 

    #Creamos un df con los datos para graficar
    roc_data = {"FPR": fpr, "TPR": tpr}
    df_roc = pd.DataFrame(roc_data)

    # Trazar la curva ROC con Plotly Express
    fig = px.line(
        df_roc,
        x="FPR",
        y="TPR",
        labels={"FPR": "Tasa de falsos positivos", "TPR": "Tasa de verdaderos positivos"},
        title="Curva ROC",
    )

    # Agregar una línea diagonal que representa el modelo aleatorio
    fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)
    fig.update_layout(
        width=800,  
        height=450
    )
    # Mostrar la figura
    fig.show()

    cols_categ = (
    [
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "MultipleLines",
        "isActive",
        "BeginDate_Year",
        "BeginDate_Month",
        "EndDate_Year",
        "EndDate_Month",
    ],
)
numeric_columns = ["MonthlyCharges", "TotalCharges"]


In [11]:
#Separacion de datos

y = full_data["isActive"]
X = full_data.drop(columns="isActive")
X = scaler_ohe(X)
X_train,X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=999)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=999)


#### LOGISTIC REGRESSION

In [25]:
Mejores_hiperparámetros = {'C': 1, 'class_weight': None, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
model_logistreg = LogisticRegression(**Mejores_hiperparámetros)

# param_grid = {
#     'C': [0.01, 0.1, 1, 10, 100],  # Regularización
#     'solver': ['liblinear', 'saga'],  # Solvers comunes para clasificación binaria
#     'class_weight': ['balanced', None],  # Considerar desbalanceo de clases o no
#     'penalty': ['l2', 'l1'],  # Tipos de regularización
#     'max_iter': [100, 200, 300]  # Número máximo de iteraciones
# }


# grid_search = GridSearchCV(estimator=model_logistreg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# grid_search.fit(X_train, y_train)

# print("Mejores hiperparámetros:", grid_search.best_params_)
# print("")
# # Predecir usando el mejor modelo encontrado
# best_model = grid_search.best_estimator_
# predictions = best_model.predict(X_test)

# #Mejores hiperparámetros: {'C': 1, 'class_weight': None, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

model_logistreg.fit(X_train, y_train)
predictions = model_logistreg.predict(X_test)
find_scores(X_test, y_test, predictions, model_logistreg)



              precision    recall  f1-score   support

           0       0.86      0.91      0.89       844
           1       0.68      0.58      0.62       283

    accuracy                           0.83      1127
   macro avg       0.77      0.74      0.75      1127
weighted avg       0.82      0.83      0.82      1127

Área bajo la curva ROC: 0.7423697519803059


#### RANDOM FOREST CLASSIFIER

In [26]:
Mejores_hiperparámetros = {'class_weight': None, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
model_rf = RandomForestClassifier(**Mejores_hiperparámetros)

# param_grid = {
#     'n_estimators': [100, 200, 300],  
#     'max_depth': [10, 20, 30],  
#     'min_samples_split': [2, 5, 10],  
#     'min_samples_leaf': [1, 2, 4],   
#     'class_weight': ['balanced', 'balanced_subsample', None]  
# }

# grid_search = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)

# print("Mejores hiperparámetros:", grid_search.best_params_)

# #Mejores hiperparámetros: {'class_weight': None, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

# # Predecir usando el mejor modelo encontrado
# best_model = grid_search.best_estimator_
# predictions = best_model.predict(X_test)

model_rf.fit(X_train,y_train)
predictions = model_rf.predict(X_test)
find_scores(X_test, y_test, predictions, model_rf)

              precision    recall  f1-score   support

           0       0.87      0.94      0.90       844
           1       0.77      0.57      0.66       283

    accuracy                           0.85      1127
   macro avg       0.82      0.76      0.78      1127
weighted avg       0.84      0.85      0.84      1127

Área bajo la curva ROC: 0.7571906452531275


#### KNN CLASSIFIER

In [27]:
Mejores_hiperparámetros = {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}

model_knn = KNeighborsClassifier(**Mejores_hiperparámetros)

# param_grid = {
#     'n_neighbors': [3, 5, 7, 9], 
#     'weights': ['uniform', 'distance'],  
#     'metric': ['euclidean', 'manhattan', 'minkowski'] 
# }

# grid_search = GridSearchCV(estimator=model_knn, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# grid_search.fit(X_train, y_train)

# print("Mejores hiperparámetros:", grid_search.best_params_)
# "Mejores hiperparámetros: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}"

# # Predecir usando el mejor modelo encontrado
# best_model = grid_search.best_estimator_
# predictions = best_model.predict(X_test)

model_knn.fit(X_train,y_train)
predictions = model_knn.predict(X_test)

find_scores(X_test, y_test, predictions, model_knn)


              precision    recall  f1-score   support

           0       0.87      0.87      0.87       844
           1       0.62      0.61      0.61       283

    accuracy                           0.81      1127
   macro avg       0.74      0.74      0.74      1127
weighted avg       0.81      0.81      0.81      1127

Área bajo la curva ROC: 0.7422650846549327


#### LIGHTGBM

In [15]:

train_data = lgbm.Dataset(X_train, label=y_train)

# # Definir el espacio de búsqueda de hiperparámetros
# param_grid = {
#     'boosting_type': ['gbdt'],
#     'objective': ['binary'],
#     'metric': ['binary_logloss'],
#     'num_leaves': [31, 50, 100],
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.1, 0.01, 0.05],
#     'min_child_samples': [20, 30],
#     'reg_alpha': [0.0, 0.1],
#     'reg_lambda': [0.0, 0.1]
# }

# # Variables para almacenar los mejores resultados
# best_params = None
# best_score = float('inf')

# # Parámetros para early stopping
# early_stopping_rounds = 10

# # Búsqueda de hiperparámetros utilizando validación cruzada
# for num_leaves in param_grid['num_leaves']:
#     for n_estimators in param_grid['num_leaves']:    
#         for learning_rate in param_grid['learning_rate']:
#                 for min_child_samples in param_grid['min_child_samples']:
#                     for reg_alpha in param_grid['reg_alpha']:
#                         for reg_lambda in param_grid['reg_lambda']:
#                             params = {
#                                 'boosting_type': 'gbdt',
#                                 'objective': 'binary',
#                                 'metric': 'binary_logloss',
#                                 'num_leaves': num_leaves,
#                                 'learning_rate': learning_rate,
#                                 'min_child_samples': min_child_samples,
#                                 'reg_alpha': reg_alpha,
#                                 'reg_lambda': reg_lambda,
#                                 'verbose': -1
#                             }

#                             # Realizar la validación cruzada
#                             cv_results = lgbm.cv(params, train_data, nfold=5, metrics='f1', stratified=True, seed=123)

#                             # Implementar early stopping manualmente
#                             min_logloss = np.min(cv_results['valid binary_logloss-mean'])
#                             best_round = np.argmin(cv_results['valid binary_logloss-mean'])

#                             if best_round + early_stopping_rounds < len(cv_results['valid binary_logloss-mean']):
#                                 continue

#                             # Verificar si es la mejor métrica encontrada
#                             if min_logloss < best_score:
#                                 best_score = min_logloss
#                                 best_params = params
                            

# # Entrenar el modelo con los mejores hiperparámetros encontrados
# print("Mejores hiperparámetros:", best_params)
"""Mejores hiperparámetros: {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 31, 'learning_rate': 0.1, 'min_child_samples': 30, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'verbose': -1}
"""


best_params = {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 31, 'learning_rate': 0.1, 'min_child_samples': 30, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'verbose': -1}


model_lgbm = lgbm.train(best_params, train_data)
predictions = ((model_lgbm.predict(X_test)) >= 0.5).astype(int)

# Evaluar el modelo
find_scores(X_test, y_test, predictions, model_lgbm)




              precision    recall  f1-score   support

           0       0.88      0.94      0.91       844
           1       0.79      0.63      0.70       283

    accuracy                           0.87      1127
   macro avg       0.84      0.79      0.81      1127
weighted avg       0.86      0.87      0.86      1127

Área bajo la curva ROC: 0.7878183980037848


"Mejores hiperparámetros: {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 31, 'learning_rate': 0.1, 'min_child_samples': 30, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'verbose': -1}\n"

#### CATBOOST

In [29]:
Mejores_hiperparámetros = {'border_count': 50, 'depth': 4, 'iterations': 600, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
model_catboost = CatBoostClassifier(**Mejores_hiperparámetros, verbose=False)

# param_grid = {
#     "iterations": [200, 400, 600],
#     "learning_rate": [0.01, 0.1, 0.2],
#     "depth": [4, 6, 8],
#     "l2_leaf_reg": [1, 3, 5],
#     "border_count": [32, 50, 100],
# }

# grid_search = GridSearchCV(estimator=model_catboost, param_grid=param_grid, scoring="roc_auc", cv=5, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# predictions = best_model.predict(X_test)

# print("Mejores hiperparámetros:", best_params)
# "Mejores hiperparámetros: {'border_count': 50, 'depth': 4, 'iterations': 600, 'l2_leaf_reg': 5, 'learning_rate': 0.1}"
model_catboost.fit(X_train, y_train)
predictions = model_catboost.predict(X_test)

find_scores(X_test,y_test, predictions, model_catboost)

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       844
           1       0.82      0.67      0.74       283

    accuracy                           0.88      1127
   macro avg       0.86      0.81      0.83      1127
weighted avg       0.88      0.88      0.88      1127

Área bajo la curva ROC: 0.8125743138010149


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Has implementado una amplia variedad de modelos, lo cual es excelente para comparar diferentes enfoques. Tu uso de GridSearchCV para la optimización de hiperparámetros es una práctica recomendada, ya que permite explorar sistemáticamente el espacio de parámetros y encontrar la mejor configuración para maximizar el rendimiento del modelo. Esto es crucial para asegurar que el modelo esté bien ajustado y no se quede atrapado en un subóptimo local.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Aunque has comentado el código de búsqueda de hiperparámetros, sería útil ver los resultados de estas búsquedas para entender cómo llegaste a los parámetros finales para cada modelo. Incluir una tabla o un resumen de los resultados de la búsqueda de hiperparámetros podría proporcionar mayor transparencia en tu proceso de modelado y ayudar a justificar por qué ciertos modelos superan a otros.
</div>


#### NEURAL NETWORK

In [17]:
model_net = Sequential()

model_net.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(16, activation='relu')) 
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(8, activation='relu')) 
model_net.add(Dropout(0.5))  # Prevención sobreajuste (Apagado aleatorio de neuronas)
model_net.add(Dense(1, activation='sigmoid'))

model_net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model_net.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=False)

loss, accuracy = model_net.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

predictions = (model_net.predict(X_test)>=0.5).astype(int)

find_scores(X_test, y_test, predictions, model_net)

c:\Users\armod\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.8198 - loss: 0.3800
Precisión en el conjunto de prueba: 0.8217
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       844
           1       0.67      0.57      0.62       283

    accuracy                           0.82      1127
   macro avg       0.77      0.74      0.75      1127
weighted avg       0.81      0.82      0.82      1127

Área bajo la curva ROC: 0.7376513489524894
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Tu implementación de la red neuronal con capas de dropout para prevenir el sobreajuste es una excelente decisión técnica. El uso de dropout ayuda a mejorar la generalización del modelo al evitar que se sobreajuste a los datos de entrenamiento, lo cual es particularmente importante en redes profundas. La estructura de la red parece bien adaptada al problema, con un balance adecuado entre la complejidad y la capacidad de modelado.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Podrías considerar experimentar con diferentes arquitecturas de red y técnicas de regularización para mejorar el rendimiento de la red neuronal, que actualmente parece estar por debajo de otros modelos. Por ejemplo, probar con más capas, cambiar la función de activación, o ajustar el learning rate podría revelar una arquitectura más eficiente para este problema. Además, técnicas como la normalización por lotes (batch normalization) o el uso de optimizadores más avanzados como Adam podrían proporcionar mejoras adicionales.
</div>


## CONCLUSIONES

### Análisis de los Resultados:
##### Random Forest (RF Classifier):

Precision (1): 0.77

Recall (1): 0.57

F1-score (1): 0.66

ROC AUC: 0.7572

El modelo RF tiene un recall bajo para la clase positiva (0.57), lo que indica que está dejando escapar muchos verdaderos positivos, es decir, está generando un número significativo de falsos negativos. Aunque la precisión es relativamente alta (0.77), este modelo podría no ser adecuado dado el alto costo de los falsos negativos.

#####LightGBM:

Precision (1): 0.79

Recall (1): 0.63

F1-score (1): 0.70

ROC AUC: 0.7878

LightGBM mejora tanto en recall (0.63) como en F1-score en comparación con Random Forest, lo que sugiere que es mejor en identificar correctamente los positivos. Sin embargo, sigue existiendo una cantidad significativa de falsos negativos. Su ROC AUC también es superior, lo que refleja un mejor equilibrio general entre las clases.

##### CatBoost:

Precision (1): 0.82

Recall (1): 0.67

F1-score (1): 0.74

ROC AUC: 0.8126

CatBoost ofrece el mejor rendimiento en términos de recall (0.67) y F1-score (0.74) para la clase positiva. Esto indica que captura más verdaderos positivos, reduciendo la cantidad de falsos negativos, lo cual es esencial dado el costo asociado. También tiene el ROC AUC más alto (0.8126), lo que sugiere que es el mejor modelo en términos de discriminación general entre clases.

##### Red Neuronal:

Precision (1): 0.67

Recall (1): 0.57

F1-score (1): 0.62

ROC AUC: 0.7377

La red neuronal muestra el peor rendimiento en recall (0.57) y F1-score (0.62) para la clase positiva, similar al Random Forest. Esto sugiere que también genera muchos falsos negativos, lo que podría ser problemático dada la situación.

##### Selección de modelo:

CatBoost parece ser el mejor modelo en este caso, ya que ofrece el mayor recall para la clase positiva (1) y el mejor equilibrio entre precisión y recall en términos de F1-score. Este modelo minimiza los falsos negativos, lo cual es crucial dada la importancia de evitar estos errores.

LightGBM también es una buena opción, con un rendimiento cercano al de CatBoost. Si la interpretabilidad del modelo o el tiempo de entrenamiento son factores importantes, LightGBM podría ser preferible, aunque el costo de algunos falsos negativos adicionales tendría que ser evaluado.

Random Forest y Red Neuronal no parecen ser las mejores opciones en este caso, dado que tienen un recall relativamente bajo para la clase positiva, lo que resultaría en un mayor número de falsos negativos.

Futuras mejoras:

Optimización de Recall:

Dado que el costo de los falsos negativos es alto, podría ser útil ajustar aún más los hiperparámetros de los modelos CatBoost o LightGBM, o ajustar el umbral de decisión para favorecer un mayor recall.

Ajuste del Umbral: 

Consideraré mover el umbral de clasificación hacia un valor menor (por ejemplo, 0.4 en lugar de 0.5) para aumentar el recall, aunque esto podría disminuir la precisión y aumentar los falsos positivos.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Has realizado un análisis exhaustivo y bien estructurado del problema de predicción de churn. Tu enfoque metódico, desde la preparación de datos hasta la evaluación de múltiples modelos, demuestra un buen entendimiento de las prácticas de machine learning. Además, tu énfasis en el recall y la minimización de falsos negativos refleja una sólida comprensión de las implicaciones de negocio, especialmente en el contexto de retención de clientes.

Para llevar tu análisis al siguiente nivel, considera lo siguiente:

- **Profundizar en el análisis de características importantes para cada modelo:** Utiliza técnicas como SHAP (SHapley Additive exPlanations) y LIME (Local Interpretable Model-agnostic Explanations) para entender mejor cómo cada característica influye en las predicciones del modelo.

- **Explorar técnicas de balanceo de clases:** Si tu conjunto de datos está desequilibrado, podrías implementar técnicas como SMOTE (Synthetic Minority Over-sampling Technique), ADASYN, o la submuestreo de la clase mayoritaria para balancear las clases y mejorar la capacidad del modelo para generalizar.

- **Ajuste del umbral de decisión:** Para optimizar el recall sin sacrificar demasiado la precisión, investiga y experimenta con métodos como la curva ROC para elegir el umbral que maximice el trade-off entre precisión y recall. Otra técnica útil es el uso de la curva de precisión-recall, especialmente en escenarios con clases desequilibradas. También podrías considerar técnicas como el "cost-sensitive learning", donde puedes ajustar el umbral de decisión en función del costo asociado a los falsos positivos y falsos negativos.

¡Sigue adelante con el excelente trabajo, estás en el camino correcto para desarrollar modelos predictivos robustos y efectivos!
</div>
